In [1]:
#from google.colab import drive
#drive.mount('/content/drive')
import sys
import os
#PATH = '/content/drive/My Drive/PPM_Stability/'
PATH = os.getcwd()
sys.path.append(PATH)

In [2]:
#!pip install xgboost==1.0.0

In [3]:
import EncoderFactory
from DatasetManager import DatasetManager
import BucketFactory

import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample


import time
import os
import sys
from sys import argv
import pickle
from collections import defaultdict

#from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
#from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
#import catboost

# from tensorflow.keras.backend import print_tensor
# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
# from keras.layers.core import Dense, Activation, Dropout
# from keras.preprocessing import sequence
# from keras.models import Sequential, Model
# from keras.layers import Dense, Embedding, Flatten, Input
# from keras.layers import LSTM
# from keras.optimizers import Nadam, RMSprop
# from keras.layers.normalization import BatchNormalization


from hyperopt import Trials, STATUS_OK, tpe, fmin, hp
import hyperopt
from hyperopt.pyll.base import scope
from hyperopt.pyll.stochastic import sample

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [4]:
def create_and_evaluate_model(args):    
    print(args)
    global trial_nr
    trial_nr += 1
    
    start = time.time()
    score = 0
    for cv_iter in range(n_splits):
        
        dt_test_prefixes = dt_prefixes[cv_iter]
        dt_train_prefixes = pd.DataFrame()
        for cv_train_iter in range(n_splits): 
            if cv_train_iter != cv_iter:
                dt_train_prefixes = pd.concat([dt_train_prefixes, dt_prefixes[cv_train_iter]], axis=0, sort=False)
                
        #Balance unbalanced data
        if balanced_data == False:
            y = dataset_manager.get_label_numeric(dt_train_prefixes)
            case_ids = dataset_manager.get_case_ids(dt_train_prefixes)

            neg_cases = [case_ids[i] for i in range(len(case_ids)) if y[i] == 0]
            pos_cases = [case_ids[i] for i in range(len(case_ids)) if y[i] == 1]

            if len(neg_cases) > len(pos_cases):
                neg_cases = resample(neg_cases, replace = False, n_samples = len(pos_cases))
            elif len(neg_cases) < len(pos_cases):
                pos_cases = resample(pos_cases, replace = False, n_samples = len(neg_cases))

            bal_data = dt_train_prefixes.loc[dt_train_prefixes[dataset_manager.case_id_col].isin(neg_cases)]
            bal_data = bal_data.append(dt_train_prefixes.loc[dt_train_prefixes[dataset_manager.case_id_col].isin(pos_cases)])
            dt_train_prefixes = bal_data
                
        dt_test_bucket = dt_test_prefixes
        dt_train_bucket = dt_train_prefixes
        
        test_y = dataset_manager.get_label_numeric(dt_test_bucket)
        train_y = dataset_manager.get_label_numeric(dt_train_bucket)

        if len(set(train_y)) < 2:
            preds = [train_y[0]] * len(relevant_test_cases_bucket)
        else:
            feature_combiner = FeatureUnion([(method, EncoderFactory.get_encoder(method, **cls_encoder_args)) for method in methods])

            if cls_method == "xgboost":
                cls = xgb.XGBClassifier(objective='binary:logistic',
                                        n_estimators=args['n_estimators'],
                                        learning_rate= args['learning_rate'],
                                        subsample=args['subsample'],
                                        max_depth=int(args['max_depth']),
                                        colsample_bytree=args['colsample_bytree'],
                                        min_child_weight=int(args['min_child_weight']),
                                        seed=random_state)
            elif cls_method == "logit":
                cls = LogisticRegression(C=2**args['C'],
                                         random_state=random_state)
            elif cls_method == "nb":
                cls = GaussianNB(var_smoothing=args["var_smoothing"])

            pipeline = Pipeline([('encoder', feature_combiner), ('scaler', MinMaxScaler()), ('cls', cls)])
            
            #print(dt_train_bucket.shape)
            #print(len(train_y))
            #print(feature_combiner.fit_transform(dt_train_bucket).shape)
            #print(feature_combiner.fit(dt_train_bucket))
            
#             if cls_method == "nb":
#                 #train_enc = pipeline["encoder"].fit_transform(dt_train_bucket)
#                 print(train_enc[0][0])
#                 #print(train_enc.shape)
#                 #for each in train_enc:
#                 #    print(each.shape)
#                 pipeline["cls"].fit(train_enc, train_y)
            
            pipeline.fit(dt_train_bucket, train_y)

            preds = pipeline.predict(dt_test_bucket)

        if balanced_data==True:
            acc = roc_auc_score(test_y, preds)
            score += acc
        else:
            acc = f1_score(test_y, preds)
            score += acc
        
        print('Accuracy:', acc)
        
    for k, v in args.items():
        fout_all.write("%s;%s;%s;%s;%s;%s;%s\n" % (trial_nr, dataset_name, cls_method, method_name, k, v, score / n_splits))   
    fout_all.write("%s;%s;%s;%s;%s;%s;%s\n" % (trial_nr, dataset_name, cls_method, method_name, "processing_time", time.time() - start, 0))   
    fout_all.flush()
    return {'loss': -score / n_splits, 'status': STATUS_OK, 'model': cls}

In [5]:
dataset_ref = "bpic2011"
params_dir = "params"
n_iter = 10
bucket_method = "prefix"
cls_encoding = "index"
cls_method = "nb"
balanced_data = False

if bucket_method == "state":
    bucket_encoding = "last"
else:
    bucket_encoding = "agg"

method_name = "%s_%s"%(bucket_method, cls_encoding)

dataset_ref_to_datasets = {
    "bpic2011": ["bpic2011_f1"],
    "bpic2015": ["bpic2015_%s_f2"%(municipality) for municipality in range(5,6)],
    "bpic2017" : ["bpic2017_accepted"],
    "bpic2012" : ["bpic2012_accepted"],
    #"insurance": ["insurance_activity", "insurance_followup"],
    "sepsis_cases": ["sepsis_cases_1"],# "sepsis_cases_2", "sepsis_cases_4"]
    "production" : ["production"],
    "hospital": ["hospital_billing_2"],
    "traffic": ["traffic_fines_1"]
}

encoding_dict = {
    "laststate": ["static", "last"],
    "agg": ["static", "agg"],
    "index": ["static", "index"],
    "combined": ["static", "last", "agg"],
    "3d" : []
}

datasets = [dataset_ref] if dataset_ref not in dataset_ref_to_datasets else dataset_ref_to_datasets[dataset_ref]
methods = encoding_dict[cls_encoding]
print(datasets)
    
train_ratio = 0.8
n_splits = 3
random_state = 22

# create results directory
if not os.path.exists(os.path.join(params_dir)):
    os.makedirs(os.path.join(params_dir))
    
for dataset_name in datasets:
    
    # read the data
    dataset_manager = DatasetManager(dataset_name)
    data = dataset_manager.read_dataset()
    data = dataset_manager.balance_data(data)

    cls_encoder_args = {'case_id_col': dataset_manager.case_id_col, 
                        'static_cat_cols': dataset_manager.static_cat_cols,
                        'static_num_cols': dataset_manager.static_num_cols, 
                        'dynamic_cat_cols': dataset_manager.dynamic_cat_cols,
                        'dynamic_num_cols': dataset_manager.dynamic_num_cols, 
                        'fillna': True}

    # determine min and max (truncated) prefix lengths
    min_prefix_length = 1
    if "traffic_fines" in dataset_name:
        max_prefix_length = 10
    else:
        max_prefix_length = 20

    # split into training and test
    print("splitting data")
    train, _ = dataset_manager.split_data_strict(data, train_ratio, split="temporal")
    train_prefixes = dataset_manager.generate_prefix_data(train, min_prefix_length, max_prefix_length)
    
    # Bucketing prefixes based on control flow
    bucketer_args = {'encoding_method':bucket_encoding, 
                     'case_id_col':dataset_manager.case_id_col, 
                     'cat_cols':[dataset_manager.activity_col], 
                     'num_cols':[], 
                     'random_state':random_state}
    if bucket_method == "cluster":
        bucketer_args["n_clusters"] = args["n_clusters"]
    bucketer = BucketFactory.get_bucketer(bucket_method, **bucketer_args)
    bucket_assignments_train = bucketer.fit_predict(train_prefixes)
    
    for bucket in set(bucket_assignments_train):
        print("Optimising %s of %s buckets" % (bucket, len(set(bucket_assignments_train))))
        
        relevant_train_cases_bucket = dataset_manager.get_indexes(train_prefixes)[bucket_assignments_train == bucket]
        dt_train_bucket = dataset_manager.get_relevant_data_by_indexes(train_prefixes, relevant_train_cases_bucket)
    
        # prepare chunks for CV
        dt_prefixes = []
        class_ratios = []
        for train_chunk, test_chunk in dataset_manager.get_stratified_split_generator(dt_train_bucket, n_splits=n_splits):
            class_ratios.append(dataset_manager.get_class_ratio(train_chunk))
            dt_prefixes.append(train_chunk)
        #del train
        
        # set up search space
        if cls_method == "xgboost":
            space = {'n_estimators': scope.int(hp.quniform('n_estimators', 1, 175, 5)),
                    'learning_rate': hp.uniform("learning_rate", 0, 5),
                     'subsample': hp.uniform("subsample", 0.5, 1),
                     'max_depth': scope.int(hp.quniform('max_depth', 1, 30, 1)),
                     'colsample_bytree': hp.uniform("colsample_bytree", 0, 1),
                     'min_child_weight': scope.int(hp.quniform('min_child_weight', 0, 6, 1))}
        elif cls_method == "logit":
            space = {'C': hp.uniform('C', -15, 15)}
        elif cls_method == "nb":
            space = {'var_smoothing': hp.choice('v', np.logspace(0, -9, 100))}


        # optimize parameters
        trial_nr = 1
        trials = Trials()
        fout_all = open(os.path.join(PATH, params_dir, "param_optim_all_trials_%s_%s_%s_bucket_%s.csv" % (cls_method, dataset_name, method_name, bucket)), "w")
        if "prefix" in method_name:
            fout_all.write("%s;%s;%s;%s;%s;%s;%s;%s\n" % ("iter", "dataset", "cls", "method", "nr_events", "param", "value", "score"))   
        else:
            fout_all.write("%s;%s;%s;%s;%s;%s;%s\n" % ("iter", "dataset", "cls", "method", "param", "value", "score"))   
        best = fmin(create_and_evaluate_model, space, algo=tpe.suggest, max_evals=n_iter, trials=trials, verbose=True)
        fout_all.close()

        # write the best parameters
        best_params = hyperopt.space_eval(space, best)
        outfile = os.path.join(PATH, params_dir, "optimal_params_%s_%s_%s_bucket_%s.pickle" % (cls_method, dataset_name, method_name, bucket))
        # write to file
        with open(outfile, "wb") as fout:
            pickle.dump(best_params, fout)


['bpic2011_f1']
splitting data
prefix
Optimising 1 of 20 buckets
{'var_smoothing': 6.579332246575682e-07}                                                                               
Accuracy:                                                                                                              
0.9694323144104803                                                                                                     
  0%|                                                                           | 0/10 [00:00<?, ?trial/s, best loss=?]

C:\Users\velmurug\Documents\full_stability\transformers\StaticTransformer.py:44: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dt_transformed[col] = 0



Accuracy:                                                                                                              
0.9719222462203023                                                                                                     
  0%|                                                                           | 0/10 [00:00<?, ?trial/s, best loss=?]

C:\Users\velmurug\Documents\full_stability\transformers\StaticTransformer.py:44: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dt_transformed[col] = 0



Accuracy:                                                                                                              
0.9650655021834061                                                                                                     
{'var_smoothing': 0.8111308307896871}                                                                                  
 10%|████▊                                           | 1/10 [00:00<00:06,  1.34trial/s, best loss: -0.9688066876047295]

...

Accuracy:                                                                                                              
0.8390243902439024                                                                                                     
 90%|███████████████████████████████████████████▏    | 9/10 [00:55<00:05,  5.91s/trial, best loss: -0.9502071443694794]

C:\Users\velmurug\Documents\full_stability\transformers\StaticTransformer.py:44: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dt_transformed[col] = 0

C:\Users\velmurug\Documents\full_stability\transformers\IndexBasedTransformer.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dt_transformed[col] = 0



Accuracy:                                                                                                              
0.8639618138424822                                                                                                     
 90%|███████████████████████████████████████████▏    | 9/10 [00:57<00:05,  5.91s/trial, best loss: -0.9502071443694794]

C:\Users\velmurug\Documents\full_stability\transformers\StaticTransformer.py:44: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dt_transformed[col] = 0

C:\Users\velmurug\Documents\full_stability\transformers\IndexBasedTransformer.py:58: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dt_transformed[col] = 0



Accuracy:                                                                                                              
0.8741092636579572                                                                                                     
100%|███████████████████████████████████████████████| 10/10 [00:59<00:00,  5.94s/trial, best loss: -0.9502071443694794]


In [6]:
# import matplotlib.pyplot as plt
# plt.plot(space["var_smoothing"])

In [7]:
# plt.plot()

In [8]:
for bucket in set(bucket_assignments_train):
    params_path = os.path.join(PATH, params_dir, "optimal_params_%s_%s_%s_bucket_%s.pickle" % 
                               (cls_method, dataset_name, method_name, bucket))
    with open(params_path, 'rb') as f:
        best_params = pickle.load(f)
    
    print ("Bucket %s" % bucket)
    print(best_params)

Bucket 1
{'var_smoothing': 2.848035868435805e-09}
Bucket 2
{'var_smoothing': 0.0015199110829529332}
Bucket 3
{'var_smoothing': 0.0005336699231206307}
Bucket 4
{'var_smoothing': 0.0008111308307896872}
Bucket 5
{'var_smoothing': 0.0008111308307896872}
Bucket 6
{'var_smoothing': 1.232846739442066e-07}
Bucket 7
{'var_smoothing': 2.310129700083158e-09}
Bucket 8
{'var_smoothing': 1.873817422860383e-08}
Bucket 9
{'var_smoothing': 5.336699231206313e-06}
Bucket 10
{'var_smoothing': 6.579332246575683e-06}
Bucket 11
{'var_smoothing': 0.0004328761281083057}
Bucket 12
{'var_smoothing': 0.0001519911082952933}
Bucket 13
{'var_smoothing': 6.579332246575682e-08}
Bucket 14
{'var_smoothing': 1e-07}
Bucket 15
{'var_smoothing': 3.511191734215127e-05}
Bucket 16
{'var_smoothing': 8.111308307896873e-08}
Bucket 17
{'var_smoothing': 0.0002848035868435802}
Bucket 18
{'var_smoothing': 2.848035868435799e-05}
Bucket 19
{'var_smoothing': 5.336699231206313e-07}
Bucket 20
{'var_smoothing': 5.336699231206302e-09}
